In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 1.3 MB/s eta 0:00:00a 0:00:01


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
import pyspark.sql.functions as F

spark = SparkSession.builder\
        .master('local[*]')\
        .appName("Example") \
        .config("spark.some.config.option","some-value") \
        .getOrCreate()

sc = spark.sparkContext

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

## по ссылке данные не скачиваются! сам в kuggle нашел , вроде такой же файл

In [4]:
df = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)
df.show(5)

+-------------------+--------+------+------------+--------+
|           datesold|postcode| price|propertyType|bedrooms|
+-------------------+--------+------+------------+--------+
|2007-02-07 00:00:00|    2607|525000|       house|       4|
|2007-02-27 00:00:00|    2906|290000|       house|       3|
|2007-03-07 00:00:00|    2905|328000|       house|       3|
|2007-03-09 00:00:00|    2905|380000|       house|       4|
|2007-03-21 00:00:00|    2906|310000|       house|       3|
+-------------------+--------+------+------------+--------+
only showing top 5 rows



## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего (1 балл)


In [6]:
from pyspark.sql.window import Window

In [8]:
windSpec = Window.partitionBy('postcode').orderBy('datesold')
data = df.withColumn('upto_current', F.avg('price').over(windSpec.rowsBetween(-10, Window.currentRow - 1)))\
    .withColumn('after_current', F.avg('price').over(windSpec.rowsBetween(Window.currentRow, 9)))\
    .withColumn('last_sold', F.last('price').over(windSpec.rowsBetween(-1, -1)))
data.show()

+-------------------+--------+-------+------------+--------+-----------------+-------------+---------+
|           datesold|postcode|  price|propertyType|bedrooms|     upto_current|after_current|last_sold|
+-------------------+--------+-------+------------+--------+-----------------+-------------+---------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|             NULL|     669050.0|     NULL|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         327000.0|     708350.0|   327000|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|         558500.0|     698350.0|   790000|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|647333.3333333334|     679350.0|   825000|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         564250.0|     742850.0|   315000|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|         509900.0|     786600.0|   292500|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|         4797

## Задание 2
Найдите среднюю цену жилья для каждого года и приджойните эти данные к таблице из задания 1. (2 балла)


*(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)*

In [11]:
list_avg_price = data.withColumn('year', F.year('datesold')).groupBy('year').agg(F.avg('price').alias('avg_price_year'))
list_avg_price.show()

+----+-----------------+
|year|   avg_price_year|
+----+-----------------+
|2007|522377.2108843537|
|2018|660701.0427682737|
|2015|626101.3426535088|
|2013|553416.2968381312|
|2014|  592653.76143905|
|2019|634184.2187725632|
|2012|552501.3692142088|
|2009|496092.0329593268|
|2016|635185.3116683726|
|2010|559564.8102893891|
|2011|566715.1102265768|
|2008|493814.1627543036|
|2017| 671880.551420392|
+----+-----------------+



In [13]:
result = data.join(list_avg_price, on = [F.year(data.datesold) == (list_avg_price.year)], how = 'left')
result.show()

+-------------------+--------+-------+------------+--------+-----------------+-------------+---------+----+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|     upto_current|after_current|last_sold|year|   avg_price_year|
+-------------------+--------+-------+------------+--------+-----------------+-------------+---------+----+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|             NULL|     669050.0|     NULL|2007|522377.2108843537|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         327000.0|     708350.0|   327000|2007|522377.2108843537|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|         558500.0|     698350.0|   790000|2007|522377.2108843537|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|647333.3333333334|     679350.0|   825000|2008|493814.1627543036|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         564250.0|     742850.0|   315000|2008|493814.1627

## Задание 3
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)

In [14]:
result= result.withColumn("unique", F.size(F.array_distinct(F.array(result.columns))))
result.show()

+-------------------+--------+-------+------------+--------+-----------------+-------------+---------+----+-----------------+------+
|           datesold|postcode|  price|propertyType|bedrooms|     upto_current|after_current|last_sold|year|   avg_price_year|unique|
+-------------------+--------+-------+------------+--------+-----------------+-------------+---------+----+-----------------+------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|             NULL|     669050.0|     NULL|2007|522377.2108843537|     9|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         327000.0|     708350.0|   327000|2007|522377.2108843537|    10|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|         558500.0|     698350.0|   790000|2007|522377.2108843537|    10|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|647333.3333333334|     679350.0|   825000|2008|493814.1627543036|    10|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         